In [49]:
from sqlalchemy.orm import sessionmaker, declarative_base
from sqlalchemy import Column, Integer, String
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy import create_engine
from sqlalchemy import text
from sqlalchemy import inspect, MetaData, select
from dotenv import load_dotenv
import os
import pymysql
import pandas as pd
#!pip install -r requirements.txt

In [24]:
# Cargo las claves del aarchivo .env
load_dotenv() 

# URL de conexion
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT", "3306")
DB_NAME = os.getenv("DB_NAME")
DATABASE_URL = f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

In [25]:
# creo un engine de SQLAlchemy
engine = create_engine(DATABASE_URL)

# creo la sesion para interactuar con la base de datos
Session = sessionmaker(bind=engine)
session = Session()

Base = declarative_base() #declaro 'Base'

In [26]:
# Test de conexion
try:
    connection = engine.connect() #me conecto y consulto algo simple
    print("Conexión exitosa a la base de datos")
except SQLAlchemyError as e:
    print(f"Error al conectar a la base de datos: {e}")
finally:
    connection.close()  # cierro la conexión


Conexión exitosa a la base de datos


In [27]:
# Crear la tabla
test_create_table_query = text("""
CREATE TABLE IF NOT EXISTS publishers(
    publisher_id INT NOT NULL,
    name VARCHAR(255) NOT NULL,
    PRIMARY KEY(publisher_id)
);
""")

with engine.connect() as connection:
    connection.execute(test_create_table_query)
    connection.commit()  # hago el commit para confirmar


In [28]:
create_authors_tables = text("""
CREATE TABLE IF NOT EXISTS authors(
    author_id INT NOT NULL,
    first_name VARCHAR(100) NOT NULL,
    middle_name VARCHAR(50) NULL,
    last_name VARCHAR(100) NULL,
    PRIMARY KEY(author_id)
);
""")
with engine.connect() as connection:
    connection.execute(create_authors_tables)
    connection.commit()  # hago el commit para confirmar

In [29]:
create_books_table = text("""
CREATE TABLE IF NOT EXISTS books(
    book_id INT NOT NULL,
    title VARCHAR(255) NOT NULL,
    total_pages INT NULL,
    rating DECIMAL(4, 2) NULL,
    isbn VARCHAR(13) NULL,
    published_date DATE,
    publisher_id INT NULL,
    PRIMARY KEY(book_id),
    CONSTRAINT fk_publisher FOREIGN KEY(publisher_id) REFERENCES publishers(publisher_id)
);
""")
create_book_authors_table = text("""
CREATE TABLE IF NOT EXISTS book_authors (
    book_id INT NOT NULL,
    author_id INT NOT NULL,
    PRIMARY KEY(book_id, author_id),
    CONSTRAINT fk_book FOREIGN KEY(book_id) REFERENCES books(book_id) ON DELETE CASCADE,
    CONSTRAINT fk_author FOREIGN KEY(author_id) REFERENCES authors(author_id) ON DELETE CASCADE
);
""")

with engine.connect() as connection:
    connection.execute(create_books_table)
    connection.execute(create_book_authors_table)
    connection.commit()  # hago el commit para confirmar

In [30]:
# Crear una conexión a la base de datos
with engine.connect() as connection:
    # Crear un inspector para la base de datos
    inspector = inspect(connection)
    
    # Obtener la lista de tablas
    tables = inspector.get_table_names()
    
    # Mostrar las tablas
    print("Tablas en la base de datos:")
    for table in tables:
        print(table)

Tablas en la base de datos:
authors
book_authors
books
publishers


In [31]:
table_of_query = 'publishers' # poner el nombre de la tabla a consultar
query = text(f'SELECT * FROM {table_of_query};') # poner consultas a ejecutar

with engine.connect() as connection:
    result = connection.execute(query) 
    for i in result : 
        print(i)
    else:
        print("No se encontraron resultados")


(1, 'O Reilly Media')
(2, 'A Book Apart')
(3, 'A K PETERS')
(4, 'Academic Press')
(5, 'Addison Wesley')
(6, 'Albert&Sweigart')
(7, 'Alfred A. Knopf')
No se encontraron resultados


In [32]:
# otra forma que encontre para usar los codigos dentro de los archivos
sql_directory = 'C:/Users/Agustín/Desktop/4Geeks/Clases/11. Intro to SQL/connecting-to-a-sql-database-project-tutorial/src/sql'
file_to_use = 'C:/Users/Agustín/Desktop/4Geeks/Clases/11. Intro to SQL/connecting-to-a-sql-database-project-tutorial/src/sql/insert.sql' 
with open(file_to_use, 'r') as sql_file:
    sql_commands = sql_file.read()
    with engine.connect() as connection:
        transaction = connection.begin()  # Comienza ala transacción
        try: 
            for command in sql_commands.split(';'):
                command = command.strip()
                if command:
                    connection.execute(text(command))
            transaction.commit()
        except Exception as e:
            transaction.rollback()
            print(f"Error: {e}")
        connection.commit()  # Hacer el commit para confirmar las ejecuciones




Error: (pymysql.err.IntegrityError) (1062, "Duplicate entry '1' for key 'PRIMARY'")
[SQL: INSERT INTO publishers(publisher_id, name) VALUES (1, 'O Reilly Media')]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


In [33]:
#consultas simples de tablas
table_of_query = 'publishers'
query = text(f'SELECT * FROM {table_of_query};')

with engine.connect() as connection:
    df_result = pd.read_sql(query,connection)
    if df_result.empty:
        print("No se encontraron resultados")
    else :
        print(df_result)

   publisher_id             name
0             1   O Reilly Media
1             2     A Book Apart
2             3       A K PETERS
3             4   Academic Press
4             5   Addison Wesley
5             6  Albert&Sweigart
6             7  Alfred A. Knopf


In [71]:
#consultas simples de tablas
table_of_query = 'books'
query = text(f'SELECT * FROM {table_of_query};')

with engine.connect() as connection:
    df_result = pd.read_sql(query,connection)
    if df_result.empty:
        print("No se encontraron resultados")
    else :
        print(df_result)

   book_id                                              title  total_pages  \
0        1        Lean Software Development: An Agile Toolkit          240   
1        2                  Facing the Intelligence Explosion           91   
2        3                                    Scala in Action          419   
3        4  Patterns of Software: Tales from the Software ...          256   
4        5                                    Anatomy Of LISP          446   
5        6               Computing machinery and intelligence           24   
6        7                       XML: Visual QuickStart Guide          269   
7        8                                       SQL Cookbook          595   
8        9  The Apollo Guidance Computer: Architecture And...          439   
9       10  Minds and Computers: An Introduction to the Ph...          222   

   rating           isbn published_date  publisher_id  
0    4.17  9780320000000     2003-05-18             5  
1    3.87           None     

In [62]:
# consultando relaciones 

# creo el objeto MetaData
metadata = MetaData()

# para ver todas las tablas de la base de datos en el objeto metadata
metadata.reflect(bind=engine)

for table_name, table in metadata.tables.items():
    for fk in table.foreign_keys:
        if fk.parent.primary_key:  # Verifica si la clave foránea también es clave primaria
            print(f"Relación Uno a Uno detectada entre {table_name} y {fk.column.table.name}")
        else:
            if not fk.parent.primary_key:  # La clave foránea no es clave primaria en la tabla secundaria
                print(f"Relación Uno a Muchos detectada entre {fk.column.table.name} (uno) y {table_name} (muchos)")
    fks = list(table.foreign_keys)
    if len(fks) == 2 and table.primary_key.columns.keys() == [fk.parent.name for fk in fks]:
        print(f"Relación Muchos a Muchos detectada entre {fks[0].column.table.name} y {fks[1].column.table.name} a través de {table_name}")

# # Consultar las relaciones 
text2 = text("""
             SELECT
             kcu.table_name AS foreign_table,
             kcu.column_name AS foreign_column,
             kcu.referenced_table_name AS primary_table,
             kcu.referenced_column_name AS primary_column
             FROM 
             information_schema.key_column_usage AS kcu
             WHERE 
             kcu.referenced_table_name IS NOT NULL
             AND kcu.constraint_schema = DATABASE();
""")
text2

# Ejecutar la consulta
with engine.connect() as connection:
    result = connection.execute(text2)
    relationships = {}
    for row in result:
        foreign_table, foreign_column, primary_table, primary_column = row
        if foreign_table not in relationships:
            relationships[foreign_table] = []
        relationships[foreign_table].append((foreign_column, primary_table, primary_column))
    
    # Mostrar relaciones
    for table, refs in relationships.items():
        for ref in refs:
            foreign_column, primary_table, primary_column = ref
            print(f"Tabla {table} tiene una columna {foreign_column} que referencia la columna {primary_column} de la tabla {primary_table}")


Relación Uno a Uno detectada entre book_authors y books
Relación Uno a Uno detectada entre book_authors y authors
Relación Muchos a Muchos detectada entre books y authors a través de book_authors
Relación Uno a Muchos detectada entre publishers (uno) y books (muchos)
Tabla book_authors tiene una columna book_id que referencia la columna book_id de la tabla books
Tabla book_authors tiene una columna author_id que referencia la columna author_id de la tabla authors
Tabla books tiene una columna publisher_id que referencia la columna publisher_id de la tabla publishers


In [72]:
### Consultas de multiples tablas

# Definir las tablas
publishers = metadata.tables['publishers']
authors = metadata.tables['authors']
books = metadata.tables['books']
book_authors = metadata.tables['book_authors']


# Crear un JOIN entre las tablas a través de la tabla de unión book_authors
join_statement = (
    book_authors
    .join(authors, book_authors.c.author_id == authors.c.author_id)
    .join(books, book_authors.c.book_id == books.c.book_id)
)

# Seleccionar las columnas
query = select(
    authors.c.first_name,
    authors.c.last_name,
    books.c.title
).select_from(join_statement)

# Ejecutar la consulta
with engine.connect() as connection:
    result = connection.execute(query)
    for row in result:
        print(row)


('Merritt', 'Eric', 'Lean Software Development: An Agile Toolkit')
('Merritt', 'Eric', 'Minds and Computers: An Introduction to the Philosophy of Artificial Intelligence')
('Linda', 'Mui', 'SQL Cookbook')
('Alecos', 'Papadatos', 'XML: Visual QuickStart Guide')
('Anthony', 'Molinaro', 'Computing machinery and intelligence')
('Anthony', 'Molinaro', 'The Apollo Guidance Computer: Architecture And Operation (Springer Praxis Books / Space Exploration)')
('David', 'Cronin', 'Anatomy Of LISP')
('Richard', 'Blum', 'Patterns of Software: Tales from the Software Community')
('Yuval', 'Harari', 'Scala in Action')
('Paul', 'Albitz', 'Facing the Intelligence Explosion')
